In [0]:
%run ./config

In [0]:
display(spark.sql("SHOW CURRENT SCHEMA"))

## 1. 노트북에서 데이터를 쿼리하고 시각화하기

In [0]:
%sql
SELECT * FROM samples.nyctaxi.trips

In [0]:
%python
display(spark.read.table("samples.nyctaxi.trips"))

Databricks visualization. Run in Databricks to view.

Databricks data profile. Run in Databricks to view.

## 데이터 시각화

* Next to the **Table** tab, click + and then click Visualization.

* In the **Visualization Type** drop-down, verify that **Bar** is selected.

* Select fare_amount for the **X column**.

* Select trip_distance for the **Y column**.

* Select Average as the aggregation type.

* Select pickup_zip as the **Group by** column.

![시각화 구성](https://docs.databricks.com/aws/en/assets/images/trip_distance-79aa375bd64d7573daf37b63f59d173a.png)

### AI Assistant 활용하기

In [0]:
%sql
-- 날짜별 요금 집계 (SQL)

In [0]:
%python
# 요일별 운행 거리 총합

## 2. 데이터 적재


In [0]:
%fs head /Volumes/demo_ykko/product_tour/vol_product_tour/region.csv

In [0]:
%sql
CREATE OR REPLACE TABLE region_from_csv AS 
SELECT * FROM read_files(
  '/Volumes/demo_ykko/product_tour/vol_product_tour/region.csv',
  format => 'csv',
  inferSchema => true,
  header => true
)

In [0]:
%sql 
SELECT * FROM region_from_csv;

## 3. Lakehouse Federation 으로 원격 데이터 조회

In [0]:
%sql
-- SQL Server 에서 실시간 데이터 조회
SELECT * FROM mssql_foreigncatalog.ak.nations LIMIT 100;


In [0]:
%sql
CREATE OR REPLACE TABLE nation_sales AS 
SELECT  n_nationkey, n_regionkey, n_name, o_orderstatus, count(distinct o_custkey) as customer_cnt, 
  sum(o_totalprice) / 1000 as sales_amount 
FROM samples.tpch.orders o
JOIN samples.tpch.customer c ON o.o_custkey = c.c_custkey
JOIN mssql_foreigncatalog.ak.nations n ON c.c_nationkey = n.n_nationkey
GROUP BY n_nationkey, n_regionkey, n_name, o_orderstatus

In [0]:
%sql
SELECT * FROM nation_sales;

In [0]:
%sql
CREATE OR REPLACE TABLE regional_sales AS
SELECT r_name as region_name, round(sum(sales_amount)) as total_sales
FROM nation_sales n
JOIN region_from_csv r ON n.n_regionkey = r.r_regionkey
GROUP BY r_name

In [0]:
%sql
SELECT * FROM regional_sales

## 5. 대시보드 (with Genie)
* Dashboard > Gallery > New York Taxi 

## 6. SQL 에서 AI 기능 사용 (AI functions)

In [0]:
%sql
SELECT * FROM samples.bakehouse.media_customer_reviews LIMIT 5;

In [0]:
%sql
SELECT review,
  ai_analyze_sentiment(review) AS sentiment
FROM samples.bakehouse.media_customer_reviews
LIMIT 10;

In [0]:
%sql
SELECT
  review,
  ai_gen(
    "한글 60자 이내로 고객의 리뷰에 대한 정중한 응답 메시지를 생성하시오. 리뷰의 불만사항을 언급하고 해결을 위해 노력하겠다는 내용을 포함하세요. 고객 리뷰 : " || review
  ) AS reply
FROM samples.bakehouse.media_customer_reviews
WHERE ai_analyze_sentiment(review) = "negative"
LIMIT 5;

## Genie space